# Create a decay dataset suitable for radioactivedecay from PyNE
This notebook creates a set of decay dataset files for radioactivedecay from the PyNE v0.7.1 decay data, which is based on the [191004 ENDSF](https://github.com/pyne/pyne/pull/1216) release.

First import the necessary modules.

In [1]:
from pyne import nucname, data
from pyne.material import Material
import pyne

import math
import numpy as np
import pandas as pd
from scipy import sparse

print("Using PyNE version:",pyne.__version__)

Using PyNE version: 0.7.1


### Create a DataFrame containing the PyNE decay data
First create a list of all the ground state (non-metastable) radionuclides in PyNE. We exclude radionuclides with undefined half-lives.

In [2]:
pyne_nonmetastable_ids = []
for z in range(1,120):
    for a in range(1,300):
        try:
            id = z*10000000+a*10000
            hl = data.half_life(id)
        except:
            continue
        if hl == float("inf"): continue  # ignore stable nuclides
        elif math.isnan(hl): continue  # ignore nuclides where the half-life is undefined half-lives
        pyne_nonmetastable_ids.append(id)
print("Total number of radionuclides:", len(pyne_nonmetastable_ids))

Total number of radionuclides: 2920


Define functions to fill a Pandas DataFrame with the decay data from PyNE.

In [3]:
def add_hyphen(name):
    """Add hypen to radionuclide name string e.g. H3 to H-3."""

    for i in range(1, len(name)):
        if not name[i].isdigit():
            continue
        name = name[:i] + '-' + name[i:]
        break
    return name

def create_rows(ids):
    """Create a list of dictionaries which will become rows of the DataFrame of decay data."""

    rows = []
    for id in ids:
        name = add_hyphen(nucname.name(id))
        Z, A = nucname.znum(id), nucname.anum(id)
        hl = data.half_life(id)
        children = list(data.decay_children(id))
        bf = []
        modes = []
        for c in children:
            bf.append(data.branch_ratio(id, c))
            cZ, cA = nucname.znum(c), nucname.anum(c)
            if Z == cZ and A == cA: modes.append('IT')
            elif Z-2 == cZ and A-4 == cA: modes.append('Alpha')
            elif Z+1 == cZ and A == cA: modes.append('Beta-')
            elif Z-1 == cZ and A == cA: modes.append('Beta+ or EC')
            else: modes.append('SF or other')
        rows.append({'Radionuclide': name, 'id': id, 'Z': Z, 'A': A, 'Half-life_s': hl,
                    'Num_decay_modes': len(children), 'Progeny': children, 'Branching_fractions': bf,
                    'Modes': modes})
    return rows

Add all the PyNE decay data to a DataFrame.

In [4]:
col_names = ['Radionuclide', 'id', 'Z', 'A', 'Half-life_s', 'Num_decay_modes', 
             'Progeny', 'Branching_fractions', 'Modes']
pyne_full = pd.DataFrame(create_rows(pyne_nonmetastable_ids), columns=col_names)
pyne_full.set_index('Radionuclide', inplace=True)
pyne_full.to_csv('pyne_full.csv', index=True)
pyne_full.head(n=10)

,id,Z,A,Half-life_s,Num_decay_modes,Progeny,Branching_fractions,Modes
Radionuclide,,,,,,,,
H-3,10030000,1,3,3.887896e+08,1,[20030000],[1.0],[Beta-]
H-6,10060000,1,6,2.016584e-23,1,[10050000],[1.0],[SF or other]
He-5,20050000,2,5,8.167164e-24,0,[],[],[]
He-6,20060000,2,6,8.067000e-01,1,[30060000],[1.0],[Beta-]
He-7,20070000,2,7,1.890547e-18,0,[],[],[]
He-8,20080000,2,8,1.191000e-01,2,"[30070000, 30080000]","[0.16, 0.84]","[SF or other, Beta-]"
He-10,20100000,2,10,3.781095e-18,1,[20090000],[1.0],[SF or other]
Li-5,30050000,3,5,1.550249e-23,0,[],[],[]
Li-8,30080000,3,8,8.399000e-01,1,[40080000],[1.0],[Beta-]


### Order the DataFrame so all progeny are located below their parent
The radionuclides in the DataFrame need to be ordered so that progeny (decay children) are always located lower than their parent. This is so the subsequent matrices that we create are lower triangular.

To achieve this we first count how many times each radioactive decay mode occurs in the dataset.

In [5]:
modes = pd.Series(np.concatenate(pyne_full.Modes))
print('Beta+ or electron capture:', modes.value_counts()['Beta+ or EC'])
print('Beta-:', modes.value_counts()['Beta-'])
print('Alpha:', modes.value_counts()['Alpha'])
#print('Isomeric Transition (IT):', a.value_counts()['IT'])
print('Spontaneous Fission or other:', modes.value_counts()['SF or other'])
print('Total number of decay modes:', pyne_full.Num_decay_modes.sum())

Beta+ or electron capture: 1143
Beta-: 1133
Alpha: 580
Spontaneous Fission or other: 1257
Total number of decay modes: 4113


We order by decreasing mass number (A), followed by decreasing atomic number (Z) (as there are more Beta+ and EC decays than Beta- decays).

In [6]:
pyne_full.sort_values(by=['A', 'Z'], inplace=True, ascending=[False, False])
pyne_full.head(n=10)

,id,Z,A,Half-life_s,Num_decay_modes,Progeny,Branching_fractions,Modes
Radionuclide,,,,,,,,
Fl-285,1142850000,114,285,0.1500,1,[1122810000],[1.0],[Alpha]
Cn-285,1122850000,112,285,34.0000,1,[1102810000],[1.0],[Alpha]
Fl-284,1142840000,114,284,0.0025,0,[],[],[]
Cn-284,1122840000,112,284,0.1010,0,[],[],[]
Cn-283,1122830000,112,283,4.0000,0,[],[],[]
Cn-282,1122820000,112,282,0.0005,0,[],[],[]
Rg-282,1112820000,111,282,0.5000,1,[1092780000],[1.0],[Alpha]
Cn-281,1122810000,112,281,0.1300,1,[1102770000],[1.0],[Alpha]
Rg-281,1112810000,111,281,26.0000,0,[],[],[]


Now it is necessary to correct the positions of the remaining radionuclides that are not ordered correctly. We do this by looping over all the radionuclides in the DataFrame, and checking if their progeny are located below. If not, the positions of the parent and progeny rows in the DataFrame are switched. This process takes a few passes until all the parents and progeny are ordered correctly.

In [7]:
nuclide_list = list(pyne_full.index)
id_list = list(pyne_full.id)
swapping = 1
while swapping >= 1:
    swaps = 0
    for parent in nuclide_list:
        for c, mode, bf in zip(pyne_full.at[parent, 'Progeny'],
                               pyne_full.at[parent, 'Modes'], 
                               pyne_full.at[parent, 'Branching_fractions']):
            if data.decay_const(c) == 0.0 or c not in id_list:
                continue
            j = nuclide_list.index(parent)
            k = id_list.index(c)
            if  j > k:
                nuclide_list[j], nuclide_list[k] = nuclide_list[k], nuclide_list[j]
                id_list[j], id_list[k] = id_list[k], id_list[j]
                pyne_full = pyne_full.reindex(index=nuclide_list)
                swaps +=1
    print('Iteration', swapping,'number of swaps:', swaps)
    swapping += 1
    if swaps == 0: swapping = 0
pyne_full.head(n=10)

Iteration 1 number of swaps: 901
Iteration 2 number of swaps: 632
Iteration 3 number of swaps: 425
Iteration 4 number of swaps: 262
Iteration 5 number of swaps: 135
Iteration 6 number of swaps: 53
Iteration 7 number of swaps: 16
Iteration 8 number of swaps: 1
Iteration 9 number of swaps: 0


,id,Z,A,Half-life_s,Num_decay_modes,Progeny,Branching_fractions,Modes
Radionuclide,,,,,,,,
Fl-285,1142850000,114,285,0.1500,1,[1122810000],[1.0],[Alpha]
Cn-285,1122850000,112,285,34.0000,1,[1102810000],[1.0],[Alpha]
Fl-284,1142840000,114,284,0.0025,0,[],[],[]
Cn-284,1122840000,112,284,0.1010,0,[],[],[]
Cn-283,1122830000,112,283,4.0000,0,[],[],[]
Cn-282,1122820000,112,282,0.0005,0,[],[],[]
Rg-282,1112820000,111,282,0.5000,1,[1092780000],[1.0],[Alpha]
Cn-281,1122810000,112,281,0.1300,1,[1102770000],[1.0],[Alpha]
Rg-281,1112810000,111,281,26.0000,0,[],[],[]


### Now make the dataset files for radioactivedecay
The process of making datasets for radioactivedecay is as follows. We first make the sparse lower triangular matrix *&Lambda;*, which captures the decay relationships and branching franctions between parents and first progeny. We then make the sparse matrix *C*, which is used in decay calculations, and from this make its inverse *C<sup>-1</sup>*.

First we define some functions used for making *&Lambda;*, *C* and *C<sup>-1</sup>*.

In [8]:
def make_lambda_mat(df):
    """Make the lambda matrix and a list of the decay constants."""

    rows = np.array([], dtype=np.int)
    cols = np.array([], dtype=np.int)
    values = np.array([], dtype=np.float)
    lambdas = []
    ln2 = np.log(2)

    nuclide_list = list(df.index)
    id_list = list(df.id)

    for parent in nuclide_list:
        j = nuclide_list.index(parent)
        rows = np.append(rows, [j])
        cols = np.append(cols, [j])
        lambd = ln2/df.at[parent, 'Half-life_s']
        values = np.append(values, -lambd)
        lambdas = np.append(lambdas, lambd)
        for progeny, bf in zip(df.at[parent, 'Progeny'], df.at[parent, 'Branching_fractions']):
            if (progeny not in id_list): continue
            i = id_list.index(progeny)
            rows = np.append(rows, [i])
            cols = np.append(cols, [j])
            values = np.append(values, [lambd*bf])

    return sparse.csc_matrix((values, (rows, cols))), lambdas

def prepare_C_inv_C(df):
    """Prepare data structures needed to make C and inv_C."""

    nuclide_list = list(df.index)
    num_nuclides = len(nuclide_list)

    rows_dict = {}
    for i in range(num_nuclides-1, -1, -1):
        a,_ = lambda_mat[:,i].nonzero()
        b = a
        for j in a: 
            if j > i: 
                b = np.unique(np.concatenate((b,rows_dict[j])))
        rows_dict[i] = b

    rows_C = np.array([], dtype=np.int)
    cols_C = np.array([], dtype=np.int)
    for i in range(0, num_nuclides):
        rows_C = np.concatenate((rows_C,rows_dict[i]))
        cols_C = np.concatenate((cols_C,np.array([i]*len(rows_dict[i]))))

    C = sparse.csc_matrix((np.array([0.0]*rows_C.size, dtype=np.float64), (rows_C, cols_C)))
    inv_C = sparse.csc_matrix((np.array([0.0]*rows_C.size, dtype=np.float64), (rows_C, cols_C)))
    
    return rows_dict, rows_C, cols_C, C, inv_C

def make_C(rows_dict, rows_C, cols_C, C, lambda_mat, df):
    """Calculate C. Report cases of radionuclides with identical or similar half-lives in the same decay chain."""

    nuclide_list = list(df.index)
    for index in range(0, rows_C.size):
        i = rows_C[index]
        j = cols_C[index]
        if i == j: C[i,i] = 1.0
        else:
            sigma = 0.0
            for k in rows_dict[j]:
                if k == i: break
                sigma += lambda_mat[i,k]*C[k,j]
            if lambda_mat[j,j]==lambda_mat[i,i]: 
                print('equal decay constants:', nuclide_list[i], nuclide_list[j])
            C[i,j] = sigma/(lambda_mat[j,j]-lambda_mat[i,i])
            if abs((lambda_mat[j,j]-lambda_mat[i,i])/lambda_mat[j,j]) < 1E-4: 
                print('rel_diff of decay constants < 1E-4:', nuclide_list[i],nuclide_list[j])
    return C

def make_inv_C(rows_dict, rows_C, cols_C, C, inv_C):
    """Calculate inv_C."""

    for index in range(0, rows_C.size):
        i = rows_C[index]
        j = cols_C[index]
        if i == j: inv_C[i,i] = 1.0
        else:
            sigma = 0.0
            for k in rows_dict[j]:
                if k == i: break
                sigma -= C[i,k]*inv_C[k,j]
            inv_C[i,j] = sigma 
    return inv_C

The process of making *&Lambda;*, *C* and *C<sup>-1</sup>* is complicated as the PyNE radionuclide decay chains include some chains where two radionuclides have identical half-lives. PyNE has [special routines](https://pyne.io/theorymanual/decay.html) to cope with this, but radioactivedecay currently does not. Fortunately these cases are limited to some fairly obscure radionuclides and which are unlikely to be relevant to most practical applications.

The following is a first pass through at making *&Lambda;* and *C*. It highlights the cases where radionuclides in the same chain have identical half-lives, and also cases where radionuclides in the same chain have similar half-lives (relative difference < 1E-4).

In [9]:
lambda_mat, lambdas = make_lambda_mat(pyne_full)
rows_dict, rows_C, cols_C, C, inv_C = prepare_C_inv_C(pyne_full)
C = make_C(rows_dict, rows_C, cols_C, C, lambda_mat, pyne_full)

equal decay constants: Os-179 Pt-183
rel_diff of decay constants < 1E-4: Os-179 Pt-183


/usr/lib/python3/dist-packages/ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in double_scalars
/usr/lib/python3/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars


equal decay constants: Re-168 Ir-172
rel_diff of decay constants < 1E-4: Re-168 Ir-172
equal decay constants: Tm-149 Lu-153
rel_diff of decay constants < 1E-4: Tm-149 Lu-153


So the chains containing <sup>183</sup>Pt, <sup>172</sup>Ir and <sup>153</sup>Lu are affected. The strategy for dealing with these to allow a comparison between radioactivedecay and PyNE is to truncate the progeny of <sup>183</sup>Pt, <sup>172</sup>Ir and <sup>153</sup>Lu in radioactivedecay. This means any decay chains calculated in radioactivedecay starting from radionuclides which are parents or upwards of <sup>183</sup>Pt, <sup>172</sup>Ir and <sup>153</sup>Lu are only correct up to the activities of these three radionuclides. The results are unaffected if starting lower down the chain from these three radionuclides.

This function finds the affected radionuclides.

In [10]:
def find_affected_radionuclides(nuclide_list, lambda_mat, nuclide):
    """Find radionuclides higher in decay chain than nuclide."""

    s1 = {nuclide_list.index(nuclide)}
    index = 0
    while index < len(nuclide_list):
        s2 = set(lambda_mat.getcol(index).indices)
        if len(s1.intersection(s2)) > 0:
            s2 = set([s for s in list(s2) if s <= index])
            if s2.issubset(s1):
                index += 1
                continue
            s1 = s2.union(s1)
            index = 0
            continue
        index +=1
    return [nuclide_list[nuclide] for nuclide in s1]

nuclide_list = list(pyne_full.index)
print('Radionuclides affected for Pt-183:', find_affected_radionuclides(nuclide_list, lambda_mat, 'Pt-183'))
print('Radionuclides affected for Ir-172:', find_affected_radionuclides(nuclide_list, lambda_mat, 'Ir-172'))
print('Radionuclides affected for Lu-153:', find_affected_radionuclides(nuclide_list, lambda_mat, 'Lu-153'))

Radionuclides affected for Pt-183: ['Po-191', 'Pt-183', 'Bi-191', 'Pb-187', 'Tl-187', 'Rn-195', 'At-195', 'Hg-183', 'Au-183']
Radionuclides affected for Ir-172: ['Pb-180', 'Hg-176', 'Tl-177', 'Pt-172', 'Ir-172']
Radionuclides affected for Lu-153: ['Lu-153', 'Ta-157']


In total decay calculations for 16 radionuclides will be affected by truncating decay chains at Pt-183, Ir-172 and Lu-153.

We now truncate the daughters of <sup>183</sup>Pt, <sup>172</sup>Ir and <sup>153</sup>Lu.

In [11]:
def truncate_chain(df, nuclide):
    """Remove progeny and decay modes for nuclide from DataFrame."""

    df.at[nuclide, 'Num_decay_modes'] = 0
    df.at[nuclide, 'Progeny'] = []
    df.at[nuclide, 'Branching_fractions'] = []
    df.at[nuclide, 'Modes'] = []
    

pyne_truncated = pyne_full.copy()
truncate_chain(pyne_truncated, 'Pt-183')
truncate_chain(pyne_truncated, 'Ir-172')
truncate_chain(pyne_truncated, 'Lu-153')
pyne_truncated.to_csv('pyne_truncated.csv', index=True)
pyne_truncated.loc[['Pt-183', 'Ir-172', 'Lu-153'], :]

,id,Z,A,Half-life_s,Num_decay_modes,Progeny,Branching_fractions,Modes
Radionuclide,,,,,,,,
Pt-183,781830000,78,183,390.0,0,[],[],[]
Ir-172,771720000,77,172,4.4,0,[],[],[]
Lu-153,711530000,71,153,0.9,0,[],[],[]


Now this is done, we can make the matrices *C* and *C<sup>-1</sup>* used by radioactivedecay.

In [12]:
lambda_mat, lambdas = make_lambda_mat(pyne_truncated)
rows_dict, rows_C, cols_C, C, inv_C = prepare_C_inv_C(pyne_truncated)
C = make_C(rows_dict, rows_C, cols_C, C, lambda_mat, pyne_truncated)
inv_C = make_inv_C(rows_dict, rows_C, cols_C, C, inv_C)

### Save the outputs

Now write output files containing *C* and *C<sup>-1</sup>* in SciPy sparse format. Write another output file containing NumPy arrays with radionuclide names, decay constants (s<sup>-1</sup>) and the days to year conversion factor. These three files are suitable for radioactive decay `v0.0.7`.


In [13]:
sparse.save_npz('./c.npz', C)
sparse.save_npz('./cinverse.npz', inv_C)

np.savez_compressed('./radionuclides_decay_consts.npz', nuclide_names=np.array(list(pyne_full.index)),
                    decay_consts=lambdas, year_conv=365.25)